In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch 
from duel_adapters import FalconDataset
from duel_adapters import PreFineTuneDatasetConfig, FineTuningDatasetConfig
from duel_adapters import DuelAdapterInferenceConfig
from duel_adapters import Inference

2023-09-27 14:39:33.530496: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-27 14:39:33.778233: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-27 14:39:35.560852: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
## Experiment 1: Have a pretuned model (unsupervised on the Attention Block)
## evaluate it and see results
# add some additional prompts

## Experiment 2: Have a finetuned model (supervised done on the MLP)
## evaluate the same as experiment 1

## Experiment 3: Combine Exp 1 + Exp 2

## Experiment 4: Use general finetuning for all the layers only SFT 

In [2]:
# Load the inference object to do all sorts of testing

inference_config = DuelAdapterInferenceConfig(
    max_new_tokens=15,
    base_model='tiiuae/falcon-7b',
    adapter_weights_paths=["checkpoints/pre_finetuning_lora/checkpoint-1840"]
)

quick_run = Inference(config=inference_config)

2023-09-27 08:47:33,234 [ERROR]: Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: anindyadeep-s. Use `wandb login --relogin` to force relogin


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2023-09-27 08:49:46,568 [INFO]: => Instantiated the base model
2023-09-27 08:49:52,760 [INFO]: => Peft model from path: checkpoints/pre_finetuning_lora/checkpoint-1840 loaded successfully
2023-09-27 08:49:52,963 [INFO]: [INFERENCE]: Model tiiuae/falcon-7b is loaded successfully with all the adapters


In [3]:
# Load the dataset and samples for doing tr

dataset = FalconDataset(
    config = PreFineTuneDatasetConfig()
)

test_dataset = dataset.export_hf_dataset(split='test')
sample_test_texts, sample_test_labels = test_dataset['text'][:3], test_dataset['label'][:3]

In [10]:
train_dataset = dataset.export_hf_dataset(split='train')
sample_train_texts = train_dataset['text'][:10]

In [11]:
sample_train_texts

["Seriously, it's been months since I investigated my credit report, and I realized that some of the information was still erroneous. The 3 credit bureaus are required to authenticate these inquiries under Sections 609 ( a ) ( 1 ) ( A ) and 611 ( a ) ( 1 ). ( A ). It is not acceptable to treat these reporting inquiries as unconfirmed information without producing proof within the legal time range. Sections 609 ( a ) ( 1 ) ( A ) and 611 ( a ) ( 1 ). ( A ), please investigate the unverifiable inquiries and delete them immediately in my report : XXXXXXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXXBureau discrepancy",
 'There are Multiple accounts on my credit report that are not mineBureau discrepancy',
 'In accordance with the Fair Credit Reporting act XXXX, the following accounts have violated my rights : 1. XXXX ( XXXX ) Account # XXXX 2. Experian ( XXXX ) Account # XXXX 15 U.S.C 1681 section 602 A. States I have the right to privacy.\n\n15 U.S.C 1681 Sec

In [5]:
# Experiment 1
# have a simple test set with 3 examples
# We will first check with the attention adapters

quick_run.quick_test(sample_test_texts, sample_test_labels)

/home/ec2-user/.conda/envs/notebook/lib/python3.11/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


2023-09-27 08:54:09,950 [INFO]: => Finished testing


Generated text:  .

15 U.S.C 1681b (
Actual text:  Improper Bureau Reporting

Generated text:  
XXXX XXXX XXXX XXXX XXXX XXXX XXXX
Actual text:  Bureau discrepancy

Generated text:  Improper reporting of information to my credit report. 
I have been
Actual text:  Improper Bureau Reporting



In [13]:
# We will add some extra prompt and see what is happening

prompt = """A text will be given which is a customer complaint. Complete the sentence by using any of these 
two sentences. Here is an example:

Example 1:

Seriously, it's been months since I investigated my credit report, and I realized that some of the information was 
still erroneous. The 3 credit bureaus are required to authenticate these inquiries under Sections 
609 ( a ) ( 1 ) ( A ) and 611 ( a ) ( 1 ). ( A ). It is not acceptable to treat these reporting inquiries as 
unconfirmed information without producing proof within the legal time range. Sections 609 ( a ) ( 1 ) ( A ) and 
611 ( a ) ( 1 ). ( A ), please investigate the unverifiable inquiries and delete them immediately in 
my report : XXXXXXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX

Bureau discrepancy


Example 2:

'In accordance with the Fair Credit Reporting act XXXX, the following accounts have violated my rights 
: 1. XXXX ( XXXX ) Account # XXXX 2. Experian ( XXXX ) Account # XXXX 15 U.S.C 1681 section 602 A. States I
have the right to privacy.\n\n15 U.S.C 1681 Section 604 A Section 2 : It also states a consumer reporting agency 
can not furnish a account without my written instructions

Improper Bureau Reporting

Now complete this customer complaint here
{complaint}
"""

# Now appending this with the test examples

sample_test_texts_with_prompts = [prompt.format(complaint=complaint) for complaint in sample_test_texts]
quick_run.quick_test(sample_test_texts_with_prompts, sample_test_labels)

2023-09-27 09:06:01,542 [INFO]: => Finished testing


Generated text:  
15 U.S.C 1681 ( a )
Actual text:  Improper Bureau Reporting

Generated text:  Example 3 : 
The following accounts are not mine.
Actual text:  Bureau discrepancy

Generated text:  
I have been a victim of identity theft and I have been a victim
Actual text:  Improper Bureau Reporting



In [14]:
### Experiment 1 fails and we did not got any kind of completion expected

del quick_run 

In [3]:
from duel_adapters import logger

# Experiment 2
# Load the inference object to do all sorts of testing

dataset = FalconDataset(config = FineTuningDatasetConfig())
test_dataset = dataset.export_hf_dataset(split='test')
sample_test_texts, sample_test_labels = test_dataset['text'][:3], test_dataset['label'][:3]

inference_config = DuelAdapterInferenceConfig(
    max_new_tokens=15,
    base_model='tiiuae/falcon-7b',
    adapter_weights_paths=["checkpoints/finetuning_lora/checkpoint-690"]
)

quick_run = Inference(config=inference_config)

2023-09-27 09:24:51,658 [ERROR]: Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anindyadeep-s. Use `wandb login --relogin` to force relogin


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2023-09-27 09:27:10,921 [INFO]: => Instantiated the base model
2023-09-27 09:28:02,035 [INFO]: => Peft model from path: checkpoints/finetuning_lora/checkpoint-690 loaded successfully
2023-09-27 09:28:02,241 [INFO]: [INFERENCE]: Model tiiuae/falcon-7b is loaded successfully with all the adapters


In [6]:
# without any additional prompts the results
logger.info("[EXPERIMENT] Without using any more external prompts")
quick_run.quick_test(
    texts = sample_test_texts, 
    labels = sample_test_labels
)

2023-09-27 09:29:26,256 [INFO]: [EXPERIMENT] Without using any more external prompts
2023-09-27 09:29:49,790 [INFO]: => Finished testing


Generated text:  
I am writing to inform you that I have been a victim of identity
Actual text:  Bureau discrepancy

Generated text:  
Actual text:  Improper Bureau Reporting

Generated text:  
Actual text:  Improper Bureau Reporting



In [7]:
instruction_prompt = """
Below will be given some text which is a customer complaint and you are required to summarize/classify
the complaint/text by choosing either of these given option

1. Improper Bureau Reporting
2. Bureau discrepancy

You are not allowed to output any other text other than Improper Bureau Reporting or Bureau discrepancy

Example: 

### Human: 'In accordance with the Fair Credit Reporting act XXXX, the following accounts have violated my rights 
: 1. XXXX ( XXXX ) Account # XXXX 2. Experian ( XXXX ) Account # XXXX 15 U.S.C 1681 section 602 A. States I
have the right to privacy.\n\n15 U.S.C 1681 Section 604 A Section 2 : It also states a consumer reporting agency 
can not furnish a account without my written instructions

### Assistant: Improper Bureau Reporting

{prompt}
"""

quick_run.quick_test(
    texts =  [instruction_prompt.format(prompt=prompt) for prompt in sample_test_texts], 
    labels = sample_test_labels
)

2023-09-27 09:30:32,974 [INFO]: => Finished testing


Generated text:  ### Assistant: Improper Bureau Reporting

This is my several Complaint
Actual text:  Bureau discrepancy

Generated text:  
Actual text:  Improper Bureau Reporting

Generated text:  
Actual text:  Improper Bureau Reporting



In [9]:
### Experiment 2 also failed in this scenerio.
# Although it started to provide some results in the first example

del quick_run

In [10]:
## Experiment 3:
### We will load both the types of adapter here and see the result with and without the external prompt

from duel_adapters import logger

dataset = FalconDataset(config = FineTuningDatasetConfig())
test_dataset = dataset.export_hf_dataset(split='test')
sample_test_texts, sample_test_labels = test_dataset['text'][:3], test_dataset['label'][:3]

inference_config = DuelAdapterInferenceConfig(
    max_new_tokens=15,
    base_model='tiiuae/falcon-7b',
    adapter_weights_paths=[
        "checkpoints/pre_finetuning_lora/checkpoint-1840",
        "checkpoints/finetuning_lora/checkpoint-690"
    ]
)

quick_run = Inference(config=inference_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2023-09-27 09:36:33,845 [INFO]: => Instantiated the base model
2023-09-27 09:36:39,969 [INFO]: => Peft model from path: checkpoints/pre_finetuning_lora/checkpoint-1840 loaded successfully
2023-09-27 09:37:31,039 [INFO]: => Peft model from path: checkpoints/finetuning_lora/checkpoint-690 loaded successfully
2023-09-27 09:37:31,237 [INFO]: [INFERENCE]: Model tiiuae/falcon-7b is loaded successfully with all the adapters


In [12]:
## Now experimenting without any additional prompt and lets see how are the results

logger.info("[EXPERIMENT] Without using any more external prompts")
quick_run.quick_test(
    texts = sample_test_texts, 
    labels = sample_test_labels
)

2023-09-27 09:39:10,251 [INFO]: [EXPERIMENT] Without using any more external prompts
2023-09-27 09:39:34,073 [INFO]: => Finished testing


Generated text:  
XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXX
Actual text:  Bureau discrepancy

Generated text:  
Actual text:  Improper Bureau Reporting

Generated text:  
Actual text:  Improper Bureau Reporting



In [13]:
instruction_prompt = """
### Instruction:
Below will be given some text which is a customer complaint and you are required to summarize/classify
the complaint/text by choosing either of these given option

1. Improper Bureau Reporting
2. Bureau discrepancy

You are not allowed to output any other text other than Improper Bureau Reporting or Bureau discrepancy

Example: 

### Human: 'In accordance with the Fair Credit Reporting act XXXX, the following accounts have violated my rights 
: 1. XXXX ( XXXX ) Account # XXXX 2. Experian ( XXXX ) Account # XXXX 15 U.S.C 1681 section 602 A. States I
have the right to privacy.\n\n15 U.S.C 1681 Section 604 A Section 2 : It also states a consumer reporting agency 
can not furnish a account without my written instructions

### Assistant: Improper Bureau Reporting

{prompt}
"""

quick_run.quick_test(
    texts =  [instruction_prompt.format(prompt=prompt) for prompt in sample_test_texts], 
    labels = sample_test_labels
)

2023-09-27 09:40:13,442 [INFO]: => Finished testing


Generated text:  ### Improper Bureau Reporting : XXXX XXXX XXXX XXXX
Actual text:  Bureau discrepancy

Generated text:  
Actual text:  Improper Bureau Reporting

Generated text:  
Actual text:  Improper Bureau Reporting



In [14]:
del quick_run

In [3]:
## Experiment 4:
### Here we will be loading single adapter which is trained with all the parts of adapter

from duel_adapters import logger

dataset = FalconDataset(config = FineTuningDatasetConfig())
test_dataset = dataset.export_hf_dataset(split='test')
sample_test_texts, sample_test_labels = test_dataset['text'][:3], test_dataset['label'][:3]

inference_config = DuelAdapterInferenceConfig(
    max_new_tokens=15,
    base_model='tiiuae/falcon-7b',
    adapter_weights_paths=[
        "checkpoints/full/checkpoint-690"
    ]
)

quick_run = Inference(config=inference_config)

2023-09-27 14:44:06,884 [ERROR]: Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anindyadeep-s. Use `wandb login --relogin` to force relogin


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2023-09-27 14:46:24,402 [INFO]: => Instantiated the base model
2023-09-27 14:47:22,614 [INFO]: => Peft model from path: checkpoints/full/checkpoint-690 loaded successfully
2023-09-27 14:47:22,813 [INFO]: [INFERENCE]: Model tiiuae/falcon-7b is loaded successfully with all the adapters


In [4]:
logger.info("[EXPERIMENT] Without using any more external prompts")
quick_run.quick_test(
    texts = sample_test_texts, 
    labels = sample_test_labels
)

2023-09-27 14:47:27,904 [INFO]: [EXPERIMENT] Without using any more external prompts
2023-09-27 14:47:52,086 [INFO]: => Finished testing


Generated text:  
I am writing to you to dispute the following items on my credit report
Actual text:  Bureau discrepancy

Generated text:  
Actual text:  Improper Bureau Reporting

Generated text:  
Actual text:  Improper Bureau Reporting



In [5]:
instruction_prompt = """
### Instruction:
Below will be given some text which is a customer complaint and you are required to summarize/classify
the complaint/text by choosing either of these given option

1. Improper Bureau Reporting
2. Bureau discrepancy

You are not allowed to output any other text other than Improper Bureau Reporting or Bureau discrepancy

Example: 

### Human: 'In accordance with the Fair Credit Reporting act XXXX, the following accounts have violated my rights 
: 1. XXXX ( XXXX ) Account # XXXX 2. Experian ( XXXX ) Account # XXXX 15 U.S.C 1681 section 602 A. States I
have the right to privacy.\n\n15 U.S.C 1681 Section 604 A Section 2 : It also states a consumer reporting agency 
can not furnish a account without my written instructions

### Assistant: Improper Bureau Reporting

{prompt}
"""

quick_run.quick_test(
    texts =  [instruction_prompt.format(prompt=prompt) for prompt in sample_test_texts], 
    labels = sample_test_labels
)

2023-09-27 14:48:32,973 [INFO]: => Finished testing


Generated text:  ### Assistant: Improper Bureau Reporting

I am a victim of identity
Actual text:  Bureau discrepancy

Generated text:  
Actual text:  Improper Bureau Reporting

Generated text:  
Actual text:  Improper Bureau Reporting

